In [3]:
# Load the data and libraries
import pandas as pd
import numpy as np

payroll = pd.read_csv('https://data.cityofnewyork.us/resource/k397-673e.csv')
print(payroll)

     fiscal_year  payroll_number                agency_name    last_name  \
0           2020              67  ADMIN FOR CHILDREN'S SVCS  KIRSHENBAUM   
1           2020              67  ADMIN FOR CHILDREN'S SVCS    ARMSTRONG   
2           2020              67  ADMIN FOR CHILDREN'S SVCS       SCHEAR   
3           2020              67  ADMIN FOR CHILDREN'S SVCS       BARTON   
4           2020              67  ADMIN FOR CHILDREN'S SVCS       AILURI   
..           ...             ...                        ...          ...   
995         2020              67  ADMIN FOR CHILDREN'S SVCS       GALVEZ   
996         2020              67  ADMIN FOR CHILDREN'S SVCS     ANDERSON   
997         2020              67  ADMIN FOR CHILDREN'S SVCS       HOLMES   
998         2020              67  ADMIN FOR CHILDREN'S SVCS        MOORE   
999         2020              67  ADMIN FOR CHILDREN'S SVCS         HALL   

     first_name mid_init        agency_start_date work_location_borough  \
0        SHE